In [ ]:
!nvidia-smi

Tue Dec  2 18:10:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   47C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import numpy as np
from numba import cuda, int32
import time

# Rozmiar bloku (liczba wątków na blok)
TPB = 32

@cuda.jit
def count_kernel(arr, histogram, exp, n):
    s_counts = cuda.shared.array((10, TPB), dtype=int32)

    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    bid = cuda.blockIdx.x

    if tid < 10:
        for i in range(TPB):
            s_counts[tid, i] = 0
    cuda.syncthreads()

    digit = -1
    if gid < n:
        val = arr[gid]
        digit = (val // exp) % 10
        s_counts[digit, tid] = 1

    cuda.syncthreads()

    if tid < 10:
        count = 0
        for i in range(TPB):
            count += s_counts[tid, i]
        histogram[bid, tid] = count

@cuda.jit
def scatter_kernel(arr, out, global_offsets, exp, n):

    s_data = cuda.shared.array(TPB, dtype=int32)
    s_ranks = cuda.shared.array(TPB, dtype=int32)

    tid = cuda.threadIdx.x
    gid = cuda.grid(1)
    bid = cuda.blockIdx.x

    val = 0
    digit = -1
    if gid < n:
        val = arr[gid]
        s_data[tid] = val
        digit = (val // exp) % 10
    else:
        s_data[tid] = -1
        digit = 10

    cuda.syncthreads()

    if tid == 0:
        temp_counts = cuda.local.array(11, dtype=int32)
        for k in range(11):
            temp_counts[k] = 0

        for i in range(TPB):
            d = -1
            if s_data[i] != -1:
                d = (s_data[i] // exp) % 10
            else:
                d = 10

            s_ranks[i] = temp_counts[d]
            temp_counts[d] += 1

    cuda.syncthreads()

    if gid < n:
        final_pos = global_offsets[digit, bid] + s_ranks[tid]
        out[final_pos] = val

def radix_sort_cuda_stable(arr):
    n = arr.size
    arr = arr.astype(np.int32)
    out = np.zeros_like(arr)

    threads_per_block = TPB
    blocks = (n + threads_per_block - 1) // threads_per_block

    d_arr = cuda.to_device(arr)
    d_out = cuda.to_device(out)

    d_hist = cuda.device_array((blocks, 11), dtype=np.int32)
    offsets_host = np.zeros((11, blocks), dtype=np.int32)

    max_val = arr.max()
    exp = 1

    while max_val // exp > 0:
        d_hist.copy_to_device(np.zeros((blocks, 11), dtype=np.int32)) # Reset
        count_kernel[blocks, threads_per_block](d_arr, d_hist, exp, n)

        hist = d_hist.copy_to_host()
        total_counts = np.sum(hist, axis=0)

        digit_starts = np.zeros(11, dtype=np.int32)
        current_sum = 0
        for d in range(10):
            digit_starts[d] = current_sum
            current_sum += total_counts[d]

        for d in range(10):
            col_cumsum = 0
            start = digit_starts[d]
            for b in range(blocks):
                offsets_host[d, b] = start + col_cumsum
                col_cumsum += hist[b, d]

        d_offsets = cuda.to_device(offsets_host)

        scatter_kernel[blocks, threads_per_block](d_arr, d_out, d_offsets, exp, n)

        d_arr, d_out = d_out, d_arr
        exp *= 10

    return d_arr.copy_to_host()



In [ ]:
import time
from pathlib import Path

def get_paths_to_data(path):
  paths = []
  for file in Path(path).glob('**/*.txt'):
    paths.append(file)
  return paths

paths = get_paths_to_data(".")
for path in paths:
    times = []
    for i in range(10):
      text = Path(path).read_text()
      arr = np.array([int(x) for x in text.split()], dtype=np.int32)
      start_time = time.time()
      sorted_arr = radix_sort_cuda_stable(arr)
      end_time= time.time()
      elapsed_time = end_time - start_time
      times.append(elapsed_time)

    avg_time = np.mean(times)
    print(f"{path},{str(path).split('.')[0].split('_')[-1]},{avg_time}")
    with open("results.csv", "a") as f:
      f.write(f"{path},{str(path).split(".")[0].split("_")[-1]},{avg_time}\n")



almost_sorted_data_100000.txt,100000,0.33749434947967527
random_data_1000.txt,1000,0.004011631011962891


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 32 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


almost_sorted_data_1000000.txt,1000000,0.7879453897476196
data_with_duplicates_100000.txt,100000,0.02597332000732422
almost_sorted_data.txt,data,0.008073830604553222
random_data_100000.txt,100000,0.08139693737030029
data_with_duplicates_1000.txt,1000,0.002241992950439453
almost_sorted_data_1000.txt,1000,0.003189873695373535
data_with_duplicates.txt,duplicates,0.0026960372924804688
random_data.txt,data,0.009513568878173829
example_data.txt,data,0.001293492317199707


/usr/local/lib/python3.12/dist-packages/numba_cuda/numba/cuda/dispatcher.py:697: NumbaPerformanceWarning: Grid size 1 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


random_data_1000000.txt,1000000,0.9049330472946167
data_with_duplicates_1000000.txt,1000000,0.30639622211456297


In [ ]:
with open("results.csv", "a") as f:
  f.write("--------------------------------------,32,---\n")

In [ ]:
def counting_sort_for_radix(arr, exp):
    n = len(arr)
    output = [0] * n
    count = [0] * 10

    for i in range(n):
        index = (arr[i] // exp) % 10
        count[index] += 1

    for i in range(1, 10):
        count[i] += count[i - 1]

    for i in range(n - 1, -1, -1):
        index = (arr[i] // exp) % 10
        output[count[index] - 1] = arr[i]
        count[index] -= 1

    for i in range(n):
        arr[i] = output[i]


def radix_sort(arr):
    if not arr.any():
        return arr

    max_val = max(arr)
    exp = 1
    while max_val // exp > 0:
        counting_sort_for_radix(arr, exp)
        exp *= 10
    return arr


In [ ]:
paths = get_paths_to_data(".")
for path in paths:
    times = []
    for i in range(10):
      text = Path(path).read_text()
      arr = np.array([int(x) for x in text.split()], dtype=np.int32)

      start_time = time.time()
      sorted_arr = radix_sort(arr)
      end_time= time.time()

      elapsed_time = end_time - start_time
      times.append(elapsed_time)

    avg_time = np.mean(times)
    print(f"{path},{str(path).split('.')[0].split('_')[-1]},{avg_time}")
    with open("results_without_cuda.csv", "a") as f:
      f.write(f"{path},{str(path).split(".")[0].split("_")[-1]},{avg_time}\n")

almost_sorted_data_100000.txt,100000,0.5678539037704468
random_data_1000.txt,1000,0.003662872314453125
almost_sorted_data_1000000.txt,1000000,6.172169518470764
data_with_duplicates_100000.txt,100000,0.21702773571014405
almost_sorted_data.txt,data,0.07053890228271484
random_data_100000.txt,100000,0.5884534835815429
data_with_duplicates_1000.txt,1000,0.00177457332611084
almost_sorted_data_1000.txt,1000,0.0027520179748535155
data_with_duplicates.txt,duplicates,0.010458278656005859
random_data.txt,data,0.03611955642700195
example_data.txt,data,4.2390823364257815e-05
random_data_1000000.txt,1000000,7.58461365699768
data_with_duplicates_1000000.txt,1000000,2.254510688781738
